# Problem 3: Drawing Regions Of Interest (ROIs)
Given the summary images, the next step is to draw ROIs around the image to isolate the pixels belonging to a specific cell.

## Part A:
Write a short script that takes in a summary image and potentially other information (i.e., a seed pixel selected by the user), and outputs a binary mask over an ROI. Find 5 different ROIs in the data provided.

In [2]:
import plotly
import plotly.graph_objects as go
import tifffile
from google.colab import drive
from PIL import Image
import numpy as np

# Find the tif file in google drive
drive.mount('/content/drive')
file = "/content/drive/MyDrive/Neural_Signals_and_Computation_HW1/TEST_MOVIE_00001-small.tif"

# Load tif file into numpy array and image
data = tifffile.imread(file)

data_max = data[0]
data_min = data[0]
for i in range(data.shape[0]):
  data_max = np.maximum(data[i], data_max)
  data_min = np.minimum(data[i], data_min)

data_range = np.subtract(data_max, data_min)

# Apply Gaussian blur to reduce noise
def gaussian_filter(image, sigma=1):
    # Define a Gaussian kernel
    kernel_size = int(6 * sigma + 1)
    kernel = np.fromfunction(
        lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x-(kernel_size-1)/2)**2 + (y-(kernel_size-1)/2)**2)/(2*sigma**2)),
        (kernel_size, kernel_size))
    kernel /= np.sum(kernel)  # Normalize the kernel

    # Convolve the image with the Gaussian kernel
    filtered_image = np.zeros_like(image, dtype=np.float64)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            filtered_image[i, j] = np.sum(image[max(0, i-(kernel_size-1)//2):min(image.shape[0], i+(kernel_size+1)//2),
                                                max(0, j-(kernel_size-1)//2):min(image.shape[1], j+(kernel_size+1)//2)] *
                                           kernel[:min(image.shape[0], i+(kernel_size+1)//2)-max(0, i-(kernel_size-1)//2),
                                                  :min(image.shape[1], j+(kernel_size+1)//2)-max(0, j-(kernel_size-1)//2)])
    return filtered_image
data_range = gaussian_filter(data_range, 3)

# Create ROI Binary Mask
data_max = np.max(data_range)
data_min = np.min(data_range)
data_threshold = (data_max - data_min) * 0.5 + data_min
for i in range(len(data_range)):
  for j in range(len(data_range[i])):
    if data_range[i][j] < data_threshold:
      data_range[i][j] = 0
    else:
      data_range[i][j] = 1

# Plot the summary images
# Create a square layout for the plot
layout = go.Layout(
    width=data.shape[1],  # Set the width of the plot
    height=data.shape[2],  # Set the height of the plot
    xaxis=dict(range=[0, data.shape[1]]),  # Set the x-axis range to match the width of the image
    yaxis=dict(range=[0, data.shape[2]]),  # Set the y-axis range to match the height of the image
    margin=dict(l=0, r=0, t=0, b=0),  # Set the margins to 0 to remove unnecessary spacing
)

fig_range = go.Figure(data=go.Heatmap(z=data_range), layout=layout)
fig_range.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
